In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Uncomment the following lines to enabled CGroups verbose logging
#logging.getLogger('cgroups').setLevel(logging.DEBUG)
#logging.getLogger('cgroups.cpuset').setLevel(logging.DEBUG)

In [2]:
import json
import operator

import devlib
import trappy
import bart

from bart.sched.SchedMultiAssert import SchedMultiAssert
from wlgen import RTA

## Global configuration

In [3]:
# Host side results folder
RESULTS_DIR = '/tmp/schedtest'

# Taerget side temporary folder
TARGET_DIR = '/root/schedtest'

# List of tools to install on the target system
TOOLS = ["rt-app", "trace-cmd", "taskset", "cgroup_run_into.sh"]

# List of modules to enable
MODULES = ['cgroups', 'bl']

## Target connection

In [4]:
from env import TestEnv

my_target_conf = {
    "platform"    : "linux",
    "board"       : "juno",
    "host"        : "192.168.0.10",
    "username"    : "root",
    "password"    : "",
    "rtapp-calib" : {
        '0': 363, '1': 138, '2': 139, '3': 352, '4': 353, '5': 361
    },
}

# Setup the required Test Environment supports
my_tests_conf = {
    # list of additional devlib modules to install 
    "modules" : ['cgroups', 'bl', 'cpufreq'],
    # list of additional binary tools to install
    "tools" : ['rt-app', 'trace-cmd', 'cgroup_run_into.sh'],
    "ftrace" : {
         "events" : [
             "sched_switch"
         ],
         "buffsize" : 10240
    }
}

env = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
t = env.target

# Report target connection
logging.info('Connected to %s target', t.abi)

06:18:54  INFO    :         Target - Using base path: /home/derkling/Code/schedtest
06:18:54  INFO    :         Target - Connecing linux target with: {'username': 'root', 'host': '192.168.0.10', 'password': ''}
06:19:00  INFO    : Available controllers: ['cpuset', 'cpu', 'memory', 'hugetlb']
06:19:01  INFO    : Controller cpuset mounted under: /sys/fs/cgroup/devlib_cpuset
06:19:04  INFO    : Controller cpu mounted under: /sys/fs/cgroup/devlib_cpu
06:19:06  INFO    : Controller memory mounted under: /sys/fs/cgroup/devlib_memory
06:19:08  INFO    : Controller hugetlb mounted under: /sys/fs/cgroup/devlib_hugetlb
06:19:08  INFO    :         Target - Initializing target workdir [/root/devlib-target]
06:19:12  INFO    : Target topology: [[0, 3, 4, 5], [1, 2]]
06:19:14  INFO    :         FTrace - Enabled events:
06:19:14  INFO    :         FTrace -   ['sched_switch']
06:19:14  INFO    :    EnergyMeter - HWMON module not enabled
06:19:14  WARNING :    EnergyMeter - Energy sampling disabled by 

## List available Controller

In [5]:
ssys = t.cgroups.list_subsystems()
for (n,h,g,e) in ssys:
    logging.info('Controller: %10s (hierarchy id: %d) has %d cgroups',
                 n, h, g)

06:19:14  INFO    : Controller:     cpuset (hierarchy id: 1) has 1 cgroups
06:19:14  INFO    : Controller:        cpu (hierarchy id: 2) has 1 cgroups
06:19:14  INFO    : Controller:     memory (hierarchy id: 3) has 1 cgroups
06:19:14  INFO    : Controller:    hugetlb (hierarchy id: 4) has 1 cgroups


## Example of CPUSET controller usage

In [6]:
# Get a reference to the CPUSet controller
cpuset = t.cgroups.controller('cpuset')

In [7]:
# Get the list of current configured CGroups for that controller
cgroups = cpuset.list_all()
logging.info('Existing CGropups:')
for cg in cgroups:
    logging.info('  %s', cg)

06:19:15  INFO    : Existing CGropups:
06:19:15  INFO    :   /


In [8]:
# Dump the configuraiton of each controller
for cgname in cgroups:
    cgroup = cpuset.cgroup(cgname)
    attrs = cgroup.get()
    cpus = attrs['cpus']
    logging.info('%s:%-15s cpus: %s', cpuset.kind, cgroup.name, cpus)
    

06:19:15  INFO    : cpuset:/               cpus: 0-5


In [9]:
# Create a LITTLE partition
cpuset_littles = cpuset.cgroup('/LITTLE')

In [10]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "cpu_exclusive": "0\r", 
    "memory_spread_page": "0\r", 
    "sched_load_balance": "1\r", 
    "cpus": "\r", 
    "effective_mems": "\r", 
    "mem_hardwall": "0\r", 
    "mem_exclusive": "0\r", 
    "memory_pressure": "0\r", 
    "effective_cpus": "\r", 
    "mems": "\r", 
    "sched_relax_domain_level": "-1\r", 
    "memory_migrate": "0\r", 
    "memory_spread_slab": "0\r"
}


In [11]:
# Tune CPUs and MEMs attributes
#   they must be initialize for the group to be usable
cpuset_littles.set(cpus=t.bl.littles, mems=0)
print "LITTLE:\n", json.dumps(cpuset_littles.get(), indent=4)

LITTLE:
{
    "cpu_exclusive": "0\r", 
    "memory_spread_page": "0\r", 
    "sched_load_balance": "1\r", 
    "cpus": "0,3-5\r", 
    "effective_mems": "0\r", 
    "mem_hardwall": "0\r", 
    "mem_exclusive": "0\r", 
    "memory_pressure": "0\r", 
    "effective_cpus": "0,3-5\r", 
    "mems": "0\r", 
    "sched_relax_domain_level": "-1\r", 
    "memory_migrate": "0\r", 
    "memory_spread_slab": "0\r"
}


In [12]:
# Define a periodic big (80%) task
task = RTA.periodic(
    period_ms=100,
    duty_cycle_pct=80,
    duration_s=5)

# Create one task per each CPU in the target
tasks={}
for tid in enumerate(t.core_names):
    tasks['task{}'.format(tid[0])] = task

# Configure RTA to run all these tasks
rtapp = RTA(t, 'simple', calibration=env.calibration())
rtapp.conf(kind='profile', params=tasks, run_dir=TARGET_DIR);

06:19:18  INFO    : Setup new workload simple
06:19:18  INFO    : Workload duration defined by longest task
06:19:18  INFO    : Default policy: SCHED_OTHER
06:19:18  INFO    : ------------------------
06:19:18  INFO    : task [task0], sched: using default policy
06:19:18  INFO    :  | loops count: 1
06:19:18  INFO    :  + phase_000001: duration 5.000000 [s] (50 loops)
06:19:18  INFO    :  |  period   100000 [us], duty_cycle  80 %
06:19:18  INFO    :  |  run_time  80000 [us], sleep_time  20000 [us]
06:19:18  INFO    : ------------------------
06:19:18  INFO    : task [task1], sched: using default policy
06:19:18  INFO    :  | loops count: 1
06:19:18  INFO    :  + phase_000001: duration 5.000000 [s] (50 loops)
06:19:18  INFO    :  |  period   100000 [us], duty_cycle  80 %
06:19:18  INFO    :  |  run_time  80000 [us], sleep_time  20000 [us]
06:19:18  INFO    : ------------------------
06:19:18  INFO    : task [task2], sched: using default policy
06:19:18  INFO    :  | loops count: 1
06:19

In [13]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=env.ftrace, cgroup=cpuset_littles.name)

06:19:24  INFO    : Executor [start]: /root/devlib-target/bin/cgroup_run_into.sh /LITTLE '/root/devlib-target/bin/rt-app /root/schedtest/simple_00.json'
06:19:42  INFO    : Pulling trace file into [.//simple_00.dat]...
06:19:45  INFO    : Executor [end]: /root/devlib-target/bin/cgroup_run_into.sh /LITTLE '/root/devlib-target/bin/rt-app /root/schedtest/simple_00.json'


In [14]:
# Check tasks residency on little clsuter
trappy.plotter.plot_trace(trace)

In [15]:
# Compute and visualize tasks residencies on LITTLE clusterh CPUs
s = SchedMultiAssert(trappy.Run(trace), env.topology, execnames="task")
residencies = s.getResidency('cluster', env.target.bl.littles, percent=True)
print json.dumps(residencies, indent=4)

{
    "1256": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "1257": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "1258": {
        "residency": 100.00000000000001, 
        "task_name": "rt-app"
    }, 
    "1259": {
        "residency": 100.00000000000001, 
        "task_name": "rt-app"
    }, 
    "1260": {
        "residency": 100.0, 
        "task_name": "rt-app"
    }, 
    "1261": {
        "residency": 100.00000000000001, 
        "task_name": "rt-app"
    }
}


In [16]:
# Assert that ALL tasks have always executed only on LITTLE cluster
s.assertResidency('cluster', env.target.bl.littles,
                  99.9, operator.ge, percent=True, rank=len(residencies))

True

## Example of CPU controller usage

In [17]:
# Get a reference to the CPU controller
cpu = t.cgroups.controller('cpu')

In [18]:
# Create a big partition on that CPUS
cpu_littles = cpu.cgroup('/LITTLE')

In [19]:
# Check the attributes available for this control group
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "stat": "throttled_time 0\r", 
    "rt_runtime_us": "0\r", 
    "shares": "1024\r", 
    "cfs_quota_us": "-1\r", 
    "rt_period_us": "1000000\r", 
    "cfs_period_us": "100000\r"
}


In [20]:
# Set a 1CPU equivalent bandwidth for that CGroup
cpu_littles.set(cfs_period_us=100000, cfs_quota_us=50000)
print "LITTLE:\n", json.dumps(cpu_littles.get(), indent=4)

LITTLE:
{
    "stat": "throttled_time 0\r", 
    "rt_runtime_us": "0\r", 
    "shares": "1024\r", 
    "cfs_quota_us": "50000\r", 
    "rt_period_us": "1000000\r", 
    "cfs_period_us": "100000\r"
}


In [21]:
# Test execution of all these tasks into the LITTLE cluster
trace = rtapp.run(ftrace=env.ftrace, cgroup=cpu_littles.name)

06:19:56  INFO    : Executor [start]: /root/devlib-target/bin/cgroup_run_into.sh /LITTLE '/root/devlib-target/bin/rt-app /root/schedtest/simple_00.json'
06:22:03  INFO    : Pulling trace file into [.//simple_00.dat]...
06:22:07  INFO    : Executor [end]: /root/devlib-target/bin/cgroup_run_into.sh /LITTLE '/root/devlib-target/bin/rt-app /root/schedtest/simple_00.json'


In [22]:
# Check tasks residency on little clsuter
trappy.plotter.plot_trace(trace)